## Analyse de la qualité des données du fichier Marketplace_Import_Mars2025

### 1. Valeurs manquantes (11,25%)
- **45 valeurs manquantes** dans la colonne `statut_retour` (11,25% des données)
- Impact : Ces valeurs doivent être imputées pour permettre une analyse complète du cycle de vie des commandes

### 2. Problème d'uniformisation de la casse
- **Identifiants avec variations de casse** : `mkt-10003` vs `MKT-10003`
- Les identifiants `marketplace_id` présentent des inconsistances de casse qui peuvent causer des problèmes lors des jointures et analyses
- Nombre de valeurs uniques affectées : 10 identifiants différents présentent cette anomalie

### 3. Format de la colonne date
- La colonne `date` est cohérente mais reste de **type object**
- Nécessite une conversion en type `datetime` pour toute analyse temporelle (tendances, saisonnalité, etc.)

### 4. Validation des colonnes catégorielles
- **Colonne `produit`** : 10 produits uniques identifiés
  - Clavier (62 occurrences)
  - Câble USB-C (47 occurrences)
  - Disque SSD (44 occurrences)
  - Écran 24 pouces (42 occurrences)
  - Enceinte Bluetooth (42 occurrences)
  - Etc.

- **Colonne `statut_retour`** : 3 statuts valides (hors valeurs manquantes)
  - Livré (246 occurrences)
  - En attente (70 occurrences)
  - Retourné (39 occurrences)

### Points positifs
- ✅ **Aucun doublon** détecté dans le dataset
- ✅ **Complétude parfaite** pour les colonnes essentielles (`marketplace_id`, `order_id_market`, `date`, `produit`, `quantite`)
- ✅ **Cohérence du format de date** : toutes les dates sont valides et convertibles"""



In [25]:
import pandas as pd
import numpy as np

In [26]:
df_marketplace = pd.read_csv('../data/raw/Marketplace_Imports_Mars2025.csv')

In [27]:
print("AUDIT DATAFRAME MARKETPLACE")

display(df_marketplace.head())

display(df_marketplace.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_marketplace.isnull().sum() / len(df_marketplace) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n---Vérification des doublons---")
print("Nombre total de doublons :", df_marketplace.duplicated().sum())
duplicated_rows = df_marketplace[df_marketplace.duplicated(keep=False)]
print(duplicated_rows.sort_values(by=df_marketplace.columns[0]))


print("\n--- Analyse des Incohérences dans les Colonnes Catégorielles ---")
cols_to_check = ['marketplace_id', 'produit', 'statut_retour']

for col in cols_to_check:
    print(f"\nColonnes : {col}")
    # Compte les valeurs uniques après avoir converti en minuscules (pour identifier les problèmes de casse)
    if df_marketplace[col].dtype == 'object':
        value_counts = df_marketplace[col].str.lower().value_counts(dropna=False)
        print(value_counts.head(10))


# Exemple d'identification des problèmes de casse dans 'marketplace_id'
print("\nExemple de marketplace_id non uniformisé :")
# Si 'mp1' et 'MP1' existent, ils apparaîtront séparément dans df_marketplace['marketplace_id'].value_counts()
print(df_marketplace['marketplace_id'].value_counts(dropna=False).head())
df_marketplace_lowId = df_marketplace['marketplace_id'].value_counts(dropna=False)
print(f"Nombre de marketplace_id non uniformes : {df_marketplace_lowId.count()}")



print("\n--- Analyse de Cohérence de la Colonne Date ---")
try:
    # Tenter la conversion en utilisant 'coerce' pour mettre les dates invalides à NaT (Not a Time)
    df_marketplace['date_temp'] = pd.to_datetime(df_marketplace['date'], errors='coerce')

    # Compter le nombre de dates qui n'ont pas pu être converties (erreurs de format)
    invalid_dates_count = df_marketplace['date_temp'].isna().sum() - df_marketplace['date'].isna().sum()

    if invalid_dates_count > 0:
        print(f"{invalid_dates_count} entrées de la colonne 'date' n'ont pas un format de date valide.")
        # Afficher les valeurs d'origine qui ont échoué la conversion
        invalid_dates = df_marketplace[df_marketplace['date_temp'].isna() & df_marketplace['date'].notna()]['date'].unique()
        print(f"Exemples de formats invalides : {invalid_dates[:5]}")
    else:
        print("La colonne 'date' semble avoir un format cohérent et valide pour la conversion en datetime.")

    df_marketplace.drop(columns=['date_temp'], inplace=True) # Supprimer la colonne temporaire

except Exception as e:
    print(f"Une erreur inattendue est survenue lors de l'analyse de la date : {e}")


AUDIT DATAFRAME MARKETPLACE


,marketplace_id,order_id_market,date,produit,quantite,statut_retour
0,MKT-10001,774,2025-10-20,Écran 24 pouces,5,Livré
1,MKT-10002,1472,2025-05-24,Clavier,1,En attente
2,MKT-10003,1273,2024-11-14,Batterie externe,5,Livré
3,MKT-10004,1539,2024-11-30,Souris,2,Livré
4,MKT-10005,1287,2025-07-21,Clavier,2,Livré


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   marketplace_id   400 non-null    object
 1   order_id_market  400 non-null    int64 
 2   date             400 non-null    object
 3   produit          400 non-null    object
 4   quantite         400 non-null    int64 
 5   statut_retour    355 non-null    object
dtypes: int64(2), object(4)
memory usage: 18.9+ KB


None


---Taux de valeurs manquantes (%)---
statut_retour    11.25
dtype: float64

---Vérification des doublons---
Nombre total de doublons : 0
Empty DataFrame
Columns: [marketplace_id, order_id_market, date, produit, quantite, statut_retour]
Index: []

--- Analyse des Incohérences dans les Colonnes Catégorielles ---

Colonnes : marketplace_id
marketplace_id
mkt-10001    1
mkt-10002    1
mkt-10003    1
mkt-10004    1
mkt-10005    1
mkt-10006    1
mkt-10007    1
mkt-10008    1
mkt-10009    1
mkt-10010    1
Name: count, dtype: int64

Colonnes : produit
produit
clavier               62
câble usb-c           47
disque ssd            44
écran 24 pouces       42
enceinte bluetooth    42
chargeur rapide       41
webcam hd             39
batterie externe      30
casque bluetooth      29
souris                24
Name: count, dtype: int64

Colonnes : statut_retour
statut_retour
livré         246
en attente     70
NaN            45
retourné       39
Name: count, dtype: int64

Exemple de marketplace_id 

## Vérification contradictoire des problèmes de casses
L'affichage au-dessus présente la liste après avoir converti des données en minuscule

In [28]:
# Compter les marketplace_id en minuscules
print("\n--- Analyse de la casse des marketplace_id ---")

# Compter combien de valeurs sont déjà entièrement en minuscules
is_lowercase = df_marketplace['marketplace_id'].dropna().str.islower()
nb_lowercase = is_lowercase.sum()
nb_total = len(df_marketplace['marketplace_id'].dropna())

print(f"Nombre de marketplace_id en minuscules : {nb_lowercase} sur {nb_total} ({nb_lowercase/nb_total*100:.2f}%)")
print(f"Nombre de marketplace_id avec majuscules : {nb_total - nb_lowercase} sur {nb_total} ({(nb_total - nb_lowercase)/nb_total*100:.2f}%)")



--- Analyse de la casse des marketplace_id ---
Nombre de marketplace_id en minuscules : 0 sur 400 (0.00%)
Nombre de marketplace_id avec majuscules : 400 sur 400 (100.00%)


In [29]:
print("\n--- Analyse Approfondie des Colonnes 'produit' et 'statut_retour' ---")
# 1. Analyse de la colonne 'produit'
print("\n Colonne : produit (TOP 12 Libellés après uniformisation de la casse)")
# Utilisation de .str.lower() pour regrouper les variations de casse (ex: 'A' vs 'a')
valeurs_produit = df_marketplace['produit'].str.lower().value_counts()
print(valeurs_produit.head(12))
print(f"Nombre total de produits uniques : {len(valeurs_produit)}")

# 2. Analyse de la colonne 'statut_retour'
print("\n Colonne : statut_retour (Tous les Libellés après uniformisation de la casse)")
# Utilisation de .str.lower() pour regrouper les variations de casse (ex: 'Ok' vs 'OK')
valeurs_statut = df_marketplace['statut_retour'].str.lower().value_counts()
print(valeurs_statut)
print(f"Nombre total de statuts uniques (N'incluant les valeurs manquantes) : {len(valeurs_statut)}")



--- Analyse Approfondie des Colonnes 'produit' et 'statut_retour' ---

 Colonne : produit (TOP 12 Libellés après uniformisation de la casse)
produit
clavier               62
câble usb-c           47
disque ssd            44
écran 24 pouces       42
enceinte bluetooth    42
chargeur rapide       41
webcam hd             39
batterie externe      30
casque bluetooth      29
souris                24
Name: count, dtype: int64
Nombre total de produits uniques : 10

 Colonne : statut_retour (Tous les Libellés après uniformisation de la casse)
statut_retour
livré         246
en attente     70
retourné       39
Name: count, dtype: int64
Nombre total de statuts uniques (N'incluant les valeurs manquantes) : 3


## Problèmes et solutions proposées

### 45 Valeurs manquantes dans la colonne statut_retour
Les 45 valeurs manquantes doivent être imputées pour être utilisables en analyse. Nous utilisons la catégorie "Aucun retour".

### Identifiants avec des variations de casse (ex: mkt-10003 vs. MKT-10003)
Uniformisation de la casse

### La colonne date est cohérente mais reste de type object
La conversion est nécessaire pour toute analyse temporelle.

In [30]:
print("--- 1. Correction de la Casse pour 'marketplace_id' ---")
# Conversion de tous les identifiants en MAJUSCULES (ou minuscules, mais l'uniformisation est clé)
df_marketplace['marketplace_id'] = df_marketplace['marketplace_id'].str.upper()
print("La colonne 'marketplace_id' est uniformisée en majuscules.")

print("\n--- 2. Imputation des Valeurs Manquantes dans 'statut_retour' ---")
# Remplacement des NaN par 'Aucun retour'
df_marketplace['statut_retour'] = df_marketplace['statut_retour'].fillna('Aucun retour')
print(f"La colonne 'statut_retour' est imputée. NaNs restants : {df_marketplace['statut_retour'].isnull().sum()}")

print("\n--- 3. Conversion de 'date' en 'datetime' ---")
# Conversion du type object en datetime64
df_marketplace['date'] = pd.to_datetime(df_marketplace['date'])
print("La colonne 'date' est convertie au format datetime.")

# Vérification finale des types de données
print("\n--- Vérification finale des types ---")
print(df_marketplace.info())

print("\n--- Vérification des doublons après nettoyage ---")
display(df_marketplace.head(12))

--- 1. Correction de la Casse pour 'marketplace_id' ---
La colonne 'marketplace_id' est uniformisée en majuscules.

--- 2. Imputation des Valeurs Manquantes dans 'statut_retour' ---
La colonne 'statut_retour' est imputée. NaNs restants : 0

--- 3. Conversion de 'date' en 'datetime' ---
La colonne 'date' est convertie au format datetime.

--- Vérification finale des types ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   marketplace_id   400 non-null    object        
 1   order_id_market  400 non-null    int64         
 2   date             400 non-null    datetime64[ns]
 3   produit          400 non-null    object        
 4   quantite         400 non-null    int64         
 5   statut_retour    400 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 18.9+ KB
None

--- Vérification d

,marketplace_id,order_id_market,date,produit,quantite,statut_retour
0,MKT-10001,774,2025-10-20,Écran 24 pouces,5,Livré
1,MKT-10002,1472,2025-05-24,Clavier,1,En attente
2,MKT-10003,1273,2024-11-14,Batterie externe,5,Livré
3,MKT-10004,1539,2024-11-30,Souris,2,Livré
4,MKT-10005,1287,2025-07-21,Clavier,2,Livré
5,MKT-10006,1787,2025-04-10,Chargeur rapide,0,En attente
6,MKT-10007,1851,2025-02-03,Écran 24 pouces,1,Livré
7,MKT-10008,696,2025-02-14,Disque SSD,2,Livré
8,MKT-10009,8,2024-11-16,Câble USB-C,1,Livré
9,MKT-10010,1194,2025-01-06,Enceinte Bluetooth,1,Livré
